In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
import joblib

In [3]:
# Load
df = pd.read_csv("/Users/tanya-mac/Desktop/Health Risk Predictor from Lifestyle & Medical History/data/processed/model_dataset.csv")
print('Shape:', df.shape)
df.head()

Shape: (97297, 33)


,Age,gender,ethnicity,education_level,income_level,employment_status,smoking_status,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,...,triglycerides,glucose_fasting,glucose_postprandial,insulin_level,hba1c,diabetes_risk,hypertension_risk,heart_disease_risk,obesity_risk,cholesterol_imbalance
0,58,Male,Asian,Highschool,Lower-Middle,Employed,Never,0,215,5.7,...,145,136,236,6.36,8.18,80,80,20,50,80
1,52,Female,White,Highschool,Middle,Employed,Former,1,143,6.7,...,30,93,150,2.00,5.63,20,50,20,20,20
2,60,Male,Hispanic,Highschool,Middle,Unemployed,Never,1,57,6.4,...,36,118,195,5.07,7.51,80,50,20,20,20
3,74,Female,Black,Highschool,Low,Retired,Never,0,49,3.4,...,140,139,253,5.28,9.03,80,20,20,80,20
4,46,Male,White,Graduate,Middle,Retired,Never,1,109,7.2,...,160,137,184,12.74,7.20,80,50,20,20,20


In [5]:
df.shape

(97297, 33)

In [7]:
# Separate features (X) and targets (y)

X = df.drop(columns=[
    'diabetes_risk',
    'hypertension_risk',
    'heart_disease_risk',
    'obesity_risk',
    'cholesterol_imbalance'
])

y = df[[
    'diabetes_risk',
    'hypertension_risk',
    'heart_disease_risk',
    'obesity_risk',
    'cholesterol_imbalance'
]]


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [11]:
df.head()

,Age,gender,ethnicity,education_level,income_level,employment_status,smoking_status,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,...,triglycerides,glucose_fasting,glucose_postprandial,insulin_level,hba1c,diabetes_risk,hypertension_risk,heart_disease_risk,obesity_risk,cholesterol_imbalance
0,58,Male,Asian,Highschool,Lower-Middle,Employed,Never,0,215,5.7,...,145,136,236,6.36,8.18,80,80,20,50,80
1,52,Female,White,Highschool,Middle,Employed,Former,1,143,6.7,...,30,93,150,2.00,5.63,20,50,20,20,20
2,60,Male,Hispanic,Highschool,Middle,Unemployed,Never,1,57,6.4,...,36,118,195,5.07,7.51,80,50,20,20,20
3,74,Female,Black,Highschool,Low,Retired,Never,0,49,3.4,...,140,139,253,5.28,9.03,80,20,20,80,20
4,46,Male,White,Graduate,Middle,Retired,Never,1,109,7.2,...,160,137,184,12.74,7.20,80,50,20,20,20


In [12]:
df['education_level'].value_counts()

education_level
Highschool      43651
Graduate        34097
Postgraduate    14552
No formal        4997
Name: count, dtype: int64

In [13]:
df['smoking_status'].value_counts()

smoking_status
Never      58218
Current    19591
Former     19488
Name: count, dtype: int64

In [14]:
df['income_level'].value_counts()

income_level
Middle          34214
Lower-Middle    24467
Upper-Middle    19334
Low             14422
High             4860
Name: count, dtype: int64

In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

# Nominal categorical features → OneHotEncoding
categorical_features_nominal = [
    'gender',
    'ethnicity',
    'employment_status',
    'smoking_status'
]

# Ordinal categorical features → OrdinalEncoding
categorical_features_ordinal = [
    'education_level',
    'income_level'
]

# Define the ordering for ordinal encode
education_order = [
    ["No formal", "Highschool", "Graduate", "Postgraduate"]
]

income_order = [
    ["Low", "Lower-Middle", "Middle", "Upper-Middle", "High"]
]

# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('ohe_cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
         categorical_features_nominal),

        ('ord_cat', OrdinalEncoder(
            categories=education_order + income_order),
         categorical_features_ordinal)
    ],
    remainder='passthrough'
)

In [20]:
# ---------- 5) Model definition ----------
base_rf = RandomForestRegressor(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

In [21]:
multi_model = MultiOutputRegressor(base_rf, n_jobs=-1)

In [22]:
pipe = Pipeline([
    ('preprocess', preprocessor),
    ('model', multi_model)
])

In [23]:
print("Fitting pipeline ...")
pipe.fit(X_train, y_train)
print("Training finished.")

Fitting pipeline ...
Training finished.


In [24]:
y_pred = pipe.predict(X_test)  # shape (n_samples, 5)

In [27]:
target_cols = [
    'diabetes_risk',
    'hypertension_risk',
    'heart_disease_risk',
    'obesity_risk',
    'cholesterol_imbalance'
]

In [29]:
# train_multioutput_regressor.py
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import os

In [31]:
metrics = {}
for i, col in enumerate(target_cols):
    mae = mean_absolute_error(y_test.iloc[:, i], y_pred[:, i])
    rmse = np.sqrt(mean_squared_error(y_test.iloc[:, i], y_pred[:, i] ))
    r2 = r2_score(y_test.iloc[:, i], y_pred[:, i])
    metrics[col] = {"MAE": mae, "RMSE": rmse, "R2": r2}
    print(f"{col} -> MAE: {mae:.3f}, RMSE: {rmse:.3f}, R2: {r2:.3f}")

# overall (mean MAE)
mean_mae = np.mean([m["MAE"] for m in metrics.values()])
print(f"Mean MAE across targets: {mean_mae:.3f}")

diabetes_risk -> MAE: 8.155, RMSE: 15.484, R2: 0.722
hypertension_risk -> MAE: 0.000, RMSE: 0.000, R2: 1.000
heart_disease_risk -> MAE: 0.021, RMSE: 0.688, R2: 0.998
obesity_risk -> MAE: 0.000, RMSE: 0.000, R2: 1.000
cholesterol_imbalance -> MAE: 0.000, RMSE: 0.000, R2: 1.000
Mean MAE across targets: 1.635


In [32]:
# ---------- 10) Convert numeric predictions to categories ----------
def categorize(pct):
    # pct assumed 0..100
    if np.isnan(pct):
        return None
    if pct < 30:
        return "Low"
    elif pct < 55:
        return "Moderate"
    elif pct < 75:
        return "High"
    else:
        return "Very High"

pred_df = pd.DataFrame(y_pred, columns=target_cols, index=X_test.index)
pred_cat_df = pred_df.applymap(categorize)


/var/folders/h2/3jh9mwk53cg6z20yjs0w4hp00000gn/T/ipykernel_2446/847723935.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pred_cat_df = pred_df.applymap(categorize)


In [33]:
# Combine true + predicted for inspection
compare_df = pd.concat([y_test.reset_index(drop=True), pred_df.reset_index(drop=True),
                        pred_cat_df.add_suffix("_cat")], axis=1)
print("\nSample predictions (first 5 rows):")
print(compare_df.head())


Sample predictions (first 5 rows):
   diabetes_risk  hypertension_risk  heart_disease_risk  obesity_risk  \
0           20.0               80.0                20.0          20.0   
1           80.0               80.0                20.0          20.0   
2           20.0               80.0                20.0          50.0   
3           20.0               50.0                20.0          20.0   
4           20.0               80.0                20.0          80.0   

   cholesterol_imbalance  diabetes_risk  hypertension_risk  \
0                   80.0           27.5               80.0   
1                   20.0           80.0               80.0   
2                   20.0           30.8               80.0   
3                   20.0           22.1               50.0   
4                   80.0           30.2               80.0   

   heart_disease_risk  obesity_risk  cholesterol_imbalance diabetes_risk_cat  \
0                20.0          20.0                   80.0              

In [35]:
# ---------- 11) Save the pipeline ----------
model_path = "/Users/tanya-mac/Desktop/Health Risk Predictor from Lifestyle & Medical History/models/multioutput_rf_pipeline.joblib"
joblib.dump(pipe, model_path)
print(f"Saved trained pipeline to {model_path}")

Saved trained pipeline to /Users/tanya-mac/Desktop/Health Risk Predictor from Lifestyle & Medical History/models/multioutput_rf_pipeline.joblib


In [34]:
# ---------- 12) Example single-patient prediction (use X_test.iloc[0]) ----------
example_X = X_test.iloc[[0]]
example_pred = pipe.predict(example_X)[0]  # array of 5 values
example_pred_pct = example_pred  # already 0..100 in dataset convention
example_pred_cat = [categorize(x) for x in example_pred_pct]

print("\nSingle example prediction:")
for name, pct, cat in zip(target_cols, example_pred_pct, example_pred_cat):
    print(f"{name}: {pct:.0f}% ({cat})")


Single example prediction:
diabetes_risk: 28% (Low)
hypertension_risk: 80% (Very High)
heart_disease_risk: 20% (Low)
obesity_risk: 20% (Low)
cholesterol_imbalance: 80% (Very High)


# using LIGHTGBM

In [37]:
!pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 1.8 MB/s  0:00:00 eta 0:00:01


In [39]:
# install libomp
!brew install libomp

# then reinstall lightgbm (so it links correctly)
!pip uninstall -y lightgbm
!pip install lightgbm


==> Auto-updating Homebrew...
Adjust how often this is run with HOMEBREW_AUTO_UPDATE_SECS or disable with
HOMEBREW_NO_AUTO_UPDATE. Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).
==> Downloading https://ghcr.io/v2/homebrew/core/portable-ruby/blobs/sha256:c6946ba2c387b47934e77c352c2056489421003ec7ddb2abf246cef2168ec140
######################################################################### 100.0%                                                      9.3%                                              28.3%                                          45.1%########                                 58.0%###################                          68.2%#######           88.5%###########################         90.5%#############################        92.1%
==> Pouring portable-ruby-3.4.7.arm64_big_sur.bottle.tar.gz
==> Homebrew collects anonymous analytics.
Read the analytics documentation (and how to opt-out) here:
  https://docs.brew.sh/Analytics
No analytics have been recorde

In [40]:
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor

base_lgb = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

multi_model = MultiOutputRegressor(base_lgb, n_jobs=-1)


In [41]:
pipe_1 = Pipeline([
    ('preprocess', preprocessor),
    ('model', multi_model)
])

In [42]:
print("Fitting pipeline ...")
pipe_1.fit(X_train, y_train)
print("Training finished.")

Fitting pipeline ...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014704 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2711
[LightGBM] [Info] Number of data points in the train set: 72972, number of used features: 39
[LightGBM] [Info] Start training from score 55.958724
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013583 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2711
[LightGBM] [Info] Number of data points in the train set: 72972, number of used features: 39
[LightGBM] [Info] Start training from score 23.910541
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014111 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[Li

In [43]:
y_pred_1 = pipe.predict(X_test)  # shape (n_samples, 5)

In [44]:
metrics = {}
for i, col in enumerate(target_cols):
    mae = mean_absolute_error(y_test.iloc[:, i], y_pred_1[:, i])
    rmse = np.sqrt(mean_squared_error(y_test.iloc[:, i], y_pred_1[:, i] ))
    r2 = r2_score(y_test.iloc[:, i], y_pred_1[:, i])
    metrics[col] = {"MAE": mae, "RMSE": rmse, "R2": r2}
    print(f"{col} -> MAE: {mae:.3f}, RMSE: {rmse:.3f}, R2: {r2:.3f}")

# overall (mean MAE)
mean_mae = np.mean([m["MAE"] for m in metrics.values()])
print(f"Mean MAE across targets: {mean_mae:.3f}")

diabetes_risk -> MAE: 8.155, RMSE: 15.484, R2: 0.722
hypertension_risk -> MAE: 0.000, RMSE: 0.000, R2: 1.000
heart_disease_risk -> MAE: 0.021, RMSE: 0.688, R2: 0.998
obesity_risk -> MAE: 0.000, RMSE: 0.000, R2: 1.000
cholesterol_imbalance -> MAE: 0.000, RMSE: 0.000, R2: 1.000
Mean MAE across targets: 1.635
